Demo of a single and 2 layer perceptron using numpy and pysyft

Based on http://iamtrask.github.io/2015/07/12/basic-python-network/

In [1]:
import syft.syft as syft
import imp
imp.reload(syft)
import numpy as np
from syft.syft import FloatTensor,SyftController
# connect -- rerun this block if you make changes to the unity back-end
#         -- the controller is not yet smart enough to retry connection
#         -- on its own
sc = SyftController(verbose=False)

In [2]:
steps = 100
print_steps = 20

Using numpy

In [3]:
#4 inupts samples
x = np.array([ [0,0,1],[1,1,1],[1,0,1],[0,1,1] ]).astype('float')#works
#x = np.array([ [0,0,1],[0,1,1],[1,0,1],[1,1,1] ])#xor gate need hidden layer
y = np.array([[0,1,1,0]]).T.astype('float')
#perceprton weights
np.random.seed(0)
w_initial = 2*np.random.random((len(x[0]),len(y[0]))).astype('float') - 1
#activation function
def sigmoid(x,deriv=False):
    if(deriv==True):
        return x*(1-x)#the derivative is sigmoid(sigmoid(x),1)
    return 1/(1+np.exp(-x))
#predicted y pre training
print("Pre training predicttion:\n",sigmoid(np.dot(x,w_initial)))
#training
w = w_initial.copy()
for j in range(steps):
#for j in range(1):
    l1 = sigmoid(np.dot(x,w))
    l1_delta = (y - l1)*sigmoid(l1,1)
    w += x.T.dot(l1_delta)
    if j%print_steps == 0:
        print("Error: ", np.average(np.abs((y - l1))))
print("Post training predicttion:\n",sigmoid(np.dot(x,w)))

Pre training predicttion:
 [[ 0.55120158]
 [ 0.67557997]
 [ 0.5752133 ]
 [ 0.6538273 ]]
Error:  0.48855890289
Error:  0.228320154665
Error:  0.156294411201
Error:  0.123828071028
Error:  0.104899857153
Post training predicttion:
 [[ 0.11356128]
 [ 0.90848621]
 [ 0.92698831]
 [ 0.09104839]]


Using pysyft

In [4]:
X = sc.FloatTensor( x )
Y = sc.FloatTensor( y )
W_initial = sc.FloatTensor( w_initial )
W = sc.FloatTensor( w_initial )
zeros = sc.FloatTensor(np.array([[0,0,0,0]]).T)#needed as we don't have mm yet
ones = sc.FloatTensor(np.array([[1,1,1,1]]).T)#needed as we don't deriv of sigmoid yet
mean = sc.FloatTensor(np.array([[1/4,1/4,1/4,1/4]]))#hack to calculate mean
zero = sc.FloatTensor(np.array([[0]]))#also for mean

In [5]:
#predicted pre training
predition=zeros.addmm(X,W)
predition.sigmoid_()
print(predition)
#training
for j in range(steps):
    L1 = zeros.addmm(X,W)
    L1.sigmoid_()
    L1_delta = (L1-Y)*(L1*(L1-ones))
    W.addmm_(X.T(),L1_delta)
    if j%print_steps == 0:
        err = (Y - L1)
        err.abs_()
        print("Error: ",zero.addmm(mean,err))
#predicted y post training
print( zeros.addmm(X,W).sigmoid_() )

0.5512016,	
0.67558,	
0.5752133,	
0.6538273,	


Error:  0.4885589,	


Error:  0.2283202,	


Error:  0.1562944,	


Error:  0.1238281,	


Error:  0.1048998,	


0.1135612,	
0.9084862,	
0.9269884,	
0.09104829,	




In [6]:
#compare predict
L1 - sc.FloatTensor(l1)

-5.960464E-08,	
5.960464E-08,	
5.960464E-08,	
-8.195639E-08,	


In [7]:
#compare weights
W - sc.FloatTensor(w)


1.430511E-06,	
-4.917383E-07,	
-4.768372E-07,	
